In [1]:
import os
os.getcwd()
os.chdir("../")
os.getcwd()

'c:\\Projects\\CODEBASE\\DeepCNN_Classifier'

In [6]:
from dataclasses import dataclass
from pathlib import Path
from typing import List

@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [7]:
from deepClassifier.constants import *
from deepClassifier.utils import read_yaml,create_directories

In [11]:
class ConfigurationManager:
    def __init__(self,
            config_filepath= CONFIG_FILE_PATH,
            parms_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.parms = read_yaml(parms_filepath)
        create_directories([self.config.artifacts_root])

    def get_prepare_callbacks_config(self)->PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])
        prepare_callbacks_config = PrepareCallbacksConfig(
                root_dir = Path(config.root_dir),
                tensorboard_root_log_dir= Path(config.tensorboard_root_log_dir),
                checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
            )
        return prepare_callbacks_config

In [12]:
from keras.backend import log
import tensorflow as tf
import time
# creating component
class PrepareCallback:
    def __init__(self,config: PrepareCallbacksConfig):
        self.config = config

    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d_%H-%M-%S")
        tb_running_log_dir =  os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}"
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
             )
    
    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

In [13]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callbacks_config()
    prepare_callbacks = PrepareCallback(config =prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()#output:list
   
except Exception as e:
    raise(e)

In [8]:
import time
time.strftime("%Y-%m-%d_%H-%M-%S")

'2022-10-15_00-12-54'